# Create dataset with all data at luh2 resolution

In [ ]:
# Libraries
import os
import xarray as xr

In [ ]:
# Directories
dir05x = '../paper_deficit/output/05_prep_other/fig_dgvm/'

---

In [ ]:
# List of dgvms
list_dgvm = ['cablepop', 'classic', 'clm', 'dlem', 'ibis', 'isam', 'jsbach', 
             'jules', 'lpjguess', 'lpjwsl', 'orchidee']
# list of dgvms and other data
list_m = ['pot', 'erb', 'walker', 'mo', 'sanderman030', 'sanderman100', 
          'sanderman200', *list_dgvm]

In [ ]:
# Empty dataset
ds = xr.Dataset()

# Add data from dgvms and other data to empty dataset
# Process datasets for each `m` in `list_m`
for m in list_m:
    # Open the datasets
    dsm = xr.open_dataset(os.path.join(dir05x, m, f'ds_{m}_luh2res.nc'))
    dsm_land = xr.open_dataset(os.path.join(dir05x, m, f'ds_{m}_land_luh2res.nc'))
    
    # Add all variables from dsm
    ds = xr.merge([ds, dsm])
    
    # Add the land_sea_mask variable
    ds[f'{m}_land'] = dsm_land['land_sea_mask']

# Drop variables ending with 'clit' in one step
ds = ds.drop_vars([var for var in ds.data_vars if var.endswith('clit')])

# Rename variables starting with 'pot' to 'ganzenmueller'
rename_dict = {var: f'ganzenmueller{var[3:]}' for var in ds.data_vars if var.startswith('pot')}
ds = ds.rename_vars(rename_dict)

In [ ]:
# Add luh2 primary land in 1700 array to dataset
ds_prim = xr.open_dataset(os.path.join(dir05x, 'luh2/ds_luh2_prim_1700.nc'))
ds['luh2_primary_1700'] = ds_prim.prim_1700

In [ ]:
# Add grid cell area array to dataset
ds_area = xr.open_dataset(os.path.join(dir05x, 'luh2/ds_luh2_grid_cell_area.nc'))
ds['grid_cell_area_ha'] = ds_area.grid_cell_area_ha

In [ ]:
# Export
ds.to_netcdf(os.path.join(dir05x, 'dgvms_and_other_luh2res.nc'), mode='w')

---